In [ ]:
# download
# !wget -O /root/data/rafic/aircrafts/data.zip https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz
# !tar -xvzf /root/data/rafic/aircrafts/data.tar.gz -C /root/data/rafic/aircrafts/

In [10]:
import collections
import json
import mmh3  # pip install mmh3

In [14]:
path_base = '/root/data/rafic/aircrafts/fgvc-aircraft-2013b'

In [15]:
dfa = pd.concat(
    pd.read_csv(
        f'{path_base}/data/images_{which}_{split}.txt',
        delimiter=' ',
        header=None,
        names=['key', which],
    )
    for split in ('train', 'val', 'test')
    for which in ('variant', 'manufacturer')
)
dfa = dfa[dfa.variant.notnull()][['key', 'variant']].merge(dfa[dfa.manufacturer.notnull()][['key', 'manufacturer']])
dfa = dfa.assign(mfg=dfa.manufacturer.apply(lambda x: mmh3.hash(x, seed=1)))
dfa = dfa.assign(split=dfa.mfg.apply(lambda x: 'train' if x % 10 < 6 else ('val' if x % 10 in {6, 7} else 'test')))
dfa.to_csv(f'{path_base}/metadata.csv', index=False)
data = collections.defaultdict(lambda: collections.defaultdict(list))

for _, row in dfa.iterrows():
    data[row.split][row.variant].append(row['key'])
    
for o in data:
    for i in data[o]:
        data[o][i] = sorted(data[o][i])
        
json.dump(data, open(f'{path_base}/metadata.json', 'w'))        

In [17]:
dfa

,key,variant,manufacturer,mfg,split
0,1025794,707-320,Boeing,591083324,train
1,1340192,707-320,Boeing,591083324,train
2,56978,707-320,Boeing,591083324,train
3,698580,707-320,Boeing,591083324,train
4,450014,707-320,Boeing,591083324,train
...,...,...,...,...,...
9995,383400,Yak-42,Yakovlev,-1199323723,val
9996,1117062,Yak-42,Yakovlev,-1199323723,val
9997,329381,Yak-42,Yakovlev,-1199323723,val
9998,523192,Yak-42,Yakovlev,-1199323723,val


## get CLIP embs

In [50]:
import clip
import torch
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)

In [46]:
def encode_image(key: str):
    key = f'{key:07d}'
    path_emb = f'{path_base}/data/embeddings/{key}.np'
    if not Path(path_emb).exists():
        path = Path(f'{path_base}/data/images/{key}.jpg')
        assert path.exists(), f"path={path} exists"
        Path(path_emb).parent.mkdir(exist_ok=True, parents=True)
        image = preprocess(Image.open(path)).unsqueeze(0).to(device)
        emb = model.encode_image(image).squeeze().detach().numpy().astype(np.float16)
        assert emb.shape == (768,)
        np.save(open(path_emb, 'wb'), emb)

In [51]:
for key in tqdm(dfa['key']):
    encode_image(key=key)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [53]:
!ls -lh $path_base/data/embeddings/ | head

total 20M
-rw-r--r-- 1 root root 1.7K Nov 21 04:00 0034309.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:18 0034958.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:25 0037511.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:32 0037512.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:15 0038598.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:26 0038626.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:09 0038671.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:29 0041419.np
-rw-r--r-- 1 root root 1.7K Nov 21 04:24 0043750.np
ls: write error: Broken pipe


In [54]:
!ls -lh $path_base/data/embeddings/ | wc -l

10001
